In [1]:
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lesechos.json
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json
!pip install feedparser
!pip install newspaper3k

--2023-02-13 14:03:56--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lesechos.json
Resolving people.irisa.fr (people.irisa.fr)... 131.254.254.107
Connecting to people.irisa.fr (people.irisa.fr)|131.254.254.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8276509 (7,9M) [application/json]
Saving to: ‘lesechos.json’

lesechos.json       100%[===================>]   7,89M  11,3MB/s    in 0,7s    

2023-02-13 14:03:57 (11,3 MB/s) - ‘lesechos.json’ saved [8276509/8276509]

--2023-02-13 14:03:57--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json
Resolving people.irisa.fr (people.irisa.fr)... 131.254.254.107
Connecting to people.irisa.fr (people.irisa.fr)|131.254.254.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347 [application/json]
Saving to: ‘sources.json’

sources.json        100%[===================>]     347  --.-KB/s    in 0s      

2023-02-13 14:03:57 (42,9 MB/s) - ‘sources.json’

In [2]:
!python3 -m spacy download fr_core_news_md

2023-02-13 14:05:07.966109: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 14:05:08.265381: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-13 14:05:08.265423: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-13 14:05:09.396963: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [109]:
import spacy
nlp = spacy.load('fr_core_news_md')

## Question 1

In [110]:
from urllib.parse import urlparse

import json
f = open('lesechos.json')
data_lesechos = json.load(f)

# (a) se premunir contre le bloquage de commande
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# (b) recuperer le fichier RSS
import feedparser as fp
url = "https://www.lemonde.fr/rss/une.xml"
data = fp.parse(url)

for item in data.entries:
    print(item.title, item.published, item.link)

f = open('sources.json')
liste_rss_flux = json.load(f)
liste_rss_flux = list(liste_rss_flux.values())


liste_new_articles = []
# In lesechos.json, links are the keys but we cannot compare these links to the link we get when we parse RSS links.
# For example, a link from lesechos.json is 
# https://investir.lesechos.fr/marches/actualites/analyse-technique-toute-poursuite-de-la-consolidation-du-contrat-cac-40-sera-limitee-1995443.php?xtor=RSS-24
# While a link from an item entry is 
# https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/la-normalisation-des-ventes-de-velo-apres-le-boom-du-covid-laisse-des-traces-dans-les-comptes-de-thule-1905607
# So we need to compare only /marches/actualites/analyse-technique-toute-poursuite-de-la-consolidation-du-contrat-cac-40-sera-limitee
liste_url_article_already_in_base = [urlparse(temp).path[:-12] for temp in list(data_lesechos.keys())]
for url in liste_rss_flux:
    data_rss = fp.parse(url)
    for item in data_rss.entries:
        link = item.link
        res = urlparse(link)
        if res.path[:-8] not in liste_url_article_already_in_base:
            liste_new_articles.append(link)
    

Réforme des retraites : les syndicats haussent le ton et prédisent une « journée noire » le 7 mars Mon, 13 Feb 2023 10:25:42 +0100 https://www.lemonde.fr/politique/article/2023/02/13/reforme-des-retraites-les-syndicats-haussent-le-ton-et-predisent-une-journee-noire-le-7-mars_6161625_823448.html
Séisme en Turquie et en Syrie : plus de 35 000 morts et 400 000 personnes évacuées de la région Mon, 13 Feb 2023 14:20:10 +0100 https://www.lemonde.fr/international/article/2023/02/13/seisme-en-turquie-et-en-syrie-le-bilan-toujours-provisoire-s-eleve-a-plus-de-35-000-morts_6161652_3210.html
Autoroutes : le trésor des concessions attire les convoitises Mon, 13 Feb 2023 05:00:12 +0100 https://www.lemonde.fr/economie/article/2023/02/13/autoroutes-le-debat-sur-les-concessions-est-relance_6161584_3234.html
Alexeï Venediktov : « La haine est entrée au sein de chaque famille russe » Mon, 13 Feb 2023 06:00:49 +0100 https://www.lemonde.fr/international/article/2023/02/13/alexei-venediktov-la-haine-est-en

## Question 2

Les balises pour le titre et le texte d'un article sur lesechos.fr ne sont pas clairement identifiables (h1 class="sc-14kwckt-6 sc-14omazk-0 jtuxZx fsUfny" pour le title par exemple). Utiliser ```newspaper``` nous simplifie la tâche : on peut accéder facilement aux élements de la page qui nous intéressent. De plus ce package est optimisé pour les articles de presse alors qu'un package comme BeautifulSoup serait plus contraignant ici.

In [111]:
import newspaper as np

url_one_article = liste_new_articles[0]

article = np.Article('https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/la-normalisation-des-ventes-de-velo-apres-le-boom-du-covid-laisse-des-traces-dans-les-comptes-de-thule-1905607')
article.download()
article.parse()

print(article.title, article.authors)
print(article.text)
print(article.top_image)

La normalisation des ventes de vélo, après le boom du Covid, laisse des traces dans les comptes de Thule []
Par Marjorie Encelot

Publié le 10 févr. 2023 à 15:31 Mis à jour le 10 févr. 2023 à 15:57

L’année 2021, pendant la crise sanitaire, a été « exceptionnelle » pour le vélo. Trop, finalement, quand à l’instar du suédois Thule Group (-17% à la Bourse de Stockholm, plus forte baisse aujourd’hui de l’indice européen Stoxx 600), on est une entreprise cotée en Bourse, qui vend des porte-vélos et des remorques pour vélo, et que la base de comparaison est historiquement élevée au moment de présenter ses comptes 2021.

Le quatrième trimestre, au cours duquel les ventes ont chuté de près de 21% à taux de change constants (+28% par rapport à 2019), a été particulièrement « challengeant », a reconnu le patron Jan Magnus Welander lors de la conférence téléphonique avec les analystes financiers, tant les trois derniers mois de 2021 avaient été « forts », aux Etats-Unis et au Canada essentiellem

## Question 3

In [113]:
from datetime import datetime

In [114]:
for url in liste_new_articles:
    article = np.Article(url)
    article.download()
    article.parse()
    temp_dict = {'title' : article.title, 'data' : article.publish_date.strftime("%m/%d/%Y, %H:%M:%S"), 'author' : article.authors, 'category' : None, 'content' : article.text, 'image_link' : article.top_image, 'image_file' : None}
    data_lesechos[url] = temp_dict


In [116]:
with open("lesechos_updated.json", "w") as outfile:
    json.dump(data_lesechos, outfile)

## Question 4

On peut télécharger l'image à partir de l'url de l'image trouvé précédemment (cf cellule précédente) puis sauvegarder cette image dans un dossier echod_db à la racine du projet.

## Question 5

We can look into social medias such as Twitter. For example, we could follow the profile of the newspaper and for an article (in case they tweet about their article) look at some statistics such as views, RT, etc...

## Question 7

**A COMPLETER**

## Question 8

In [253]:
data_lesechos_updated = json.load(open('lesechos_updated.json'))
list_text = [temp['content'] for temp in list(data_lesechos_updated.values())]
nlp = spacy.load('fr_core_news_md', disable=['tagging', 'parser'])
docs = list(nlp.pipe(list_text))
list_entites_per_doc = []
for doc in docs:
    temp = []
    for entity in doc.ents:
        temp.append({'text' : str(entity), 'label' : str(entity.label_), 'start' : str(entity.start), 'end' : str(entity.end)})
    list_entites_per_doc.append(temp)


## Question 9

In [235]:
list_entites_per_doc

[[{'text': Kate Abnett, 'label': 'PER', 'start': 1, 'end': 3},
  {'text': Simon Jessop, 'label': 'PER', 'start': 4, 'end': 6},
  {'text': Reuters, 'label': 'ORG', 'start': 10, 'end': 11},
  {'text': Union européenne, 'label': 'ORG', 'start': 14, 'end': 16},
  {'text': Etats, 'label': 'LOC', 'start': 66, 'end': 67},
  {'text': Union européenne, 'label': 'ORG', 'start': 126, 'end': 128},
  {'text': Commission, 'label': 'ORG', 'start': 151, 'end': 152},
  {'text': Reuters, 'label': 'ORG', 'start': 155, 'end': 156},
  {'text': CO², 'label': 'MISC', 'start': 236, 'end': 237},
  {'text': Commission, 'label': 'ORG', 'start': 276, 'end': 277},
  {'text': La France, 'label': 'LOC', 'start': 301, 'end': 303},
  {'text': UE, 'label': 'ORG', 'start': 331, 'end': 332},
  {'text': Allemagne, 'label': 'LOC', 'start': 341, 'end': 342},
  {'text': accident de Fukushima, 'label': 'MISC', 'start': 352, 'end': 355},
  {'text': Autriche, 'label': 'LOC', 'start': 361, 'end': 362},
  {'text': Fonds mondial, 

In [248]:
import itertools

flat_list = list(itertools.chain(*list_entites_per_doc))

In [251]:
from collections import defaultdict

def get_most_frequent(lst, n):
    label_counts = defaultdict(int)
    for item in lst:
        label_counts[(item['text'], item['label'])] += 1

    result = defaultdict(list)
    for key, count in label_counts.items():
        result[key[1]].append((count, key[0]))

    # for label, items in result.items():
    #     items.sort(reverse=True)
    #     result[label] = items[:n]

    return result

In [252]:
get_most_frequent(flat_list, 20)

defaultdict(list,
            {'PER': [(1, Kate Abnett),
              (1, Simon Jessop),
              (1, Reportage Kate Abnett),
              (1, Sabine Siebold),
              (1, Laetitia Volga),
              (1, Stephen Culp),
              (1, Echo Wang),
              (1, Ryan Detrick),
              (1, Omicron),
              (1, Camille Raynaud),
              (1, Dow Jones),
              (1, Omicron),
              (1, Chris Beauchamp),
              (1, Hermès),
              (1, Charalambos Pissouros),
              (1, Ashitha Shivaprasad),
              (1, Norbert Rücker),
              (1, Julius Baer),
              (1, Brent),
              (1, Omicron),
              (1, Edward Moya),
              (1, Suvro Sarkar),
              (1, Reportage Ashitha Shivaprasad),
              (1, Marc Angrand),
              (1, Sophie Louet),
              (1, Pivot),
              (1, Maylis Jouaret),
              (1, Reportage Ryan Woo),
              (1, Crossley),
    

In [224]:
from collections import defaultdict

def get_most_frequent(lst, n):
    label_counts = defaultdict(int)
    for item in lst:
        for key, value in item.items():
            label_counts[(key, value['label'])] += 1

    result = defaultdict(list)
    for key, count in label_counts.items():
        result[key[1]].append((count, key))

    # for label, items in result.items():
    #     items.sort(reverse=True)
    #     result[label] = items[:n]

    return result

In [226]:
result = get_most_frequent(list_entites_per_doc, 20)

In [231]:
for label, items in result.items():
    print(items)

[(1, (Kate Abnett, 'PER')), (1, (Stephen Culp, 'PER')), (1, (Ashitha Shivaprasad, 'PER')), (1, (Simon Jessop, 'PER')), (1, (Echo Wang, 'PER')), (1, (Dow Jones, 'PER')), (1, (Pivot, 'PER')), (1, (Bloomberg, 'PER')), (1, (Bloomberg, 'PER')), (1, (Reportage Ryan Woo, 'PER')), (1, (Crossley, 'PER')), (1, (Camille Raynaud, 'PER')), (1, (Omicron, 'PER')), (1, (Covid, 'PER')), (1, (Ryan Detrick, 'PER')), (1, (Norbert Rücker, 'PER')), (1, (Julius Baer, 'PER')), (1, (Joe Biden, 'PER')), (1, (Brent, 'PER')), (1, (Maylis Jouaret, 'PER')), (1, (Anthony Fauci, 'PER')), (1, (Omicron, 'PER')), (1, (Randal Quarles, 'PER')), (1, (Omicron, 'PER')), (1, (Teleperformance, 'PER')), (1, (Alphavalue, 'PER')), (1, (Sarah Bloom Raskin, 'PER')), (1, (Joe Biden, 'PER')), (1, (Chris Beauchamp, 'PER')), (1, (Edward Moya, 'PER')), (1, (Scott Chronert, 'PER')), (1, (Omicron, 'PER')), (1, (Anthony Fauci, 'PER')), (1, (Hermès, 'PER')), (1, (Omicron, 'PER')), (1, (Reportage Kate Abnett, 'PER')), (1, (Sabine Siebold, 'P